In [1]:
import time
import os
import sys
sys.path.append(r'C:\Users\User\Documents\Code\blimp')
from utils.parse_markpoints import ParseMarkpoints
from utils.prairie_interface import PrairieInterface
from utils import mat_loader as ml
from sdk.slm_sdk import SLMsdk
import scipy.io
import yaml
import numpy as np
import imageio
import tifffile
import matplotlib.pyplot as plt
from threading import Thread
%matplotlib inline

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
laser_power = 320

In [3]:
# output sampling rate of naparm
naparm_rate = 1000

naparm_path = r'F:\Data\jrowland\2019-03-08\naparm\2019-03-08_plasticSlide_NAPARM_008'

with open(r'C:\Users\User\Documents\Code\blimp\blimp_settings.yaml', 'r') as stream:
            yaml_dict = yaml.load(stream)


In [4]:
for file in os.listdir(naparm_path):
    if file.endswith('.xml'):
        xml_path= os.path.join(naparm_path, file)
    elif file.endswith('gpl'):
        gpl_path = os.path.join(naparm_path, file)
    elif file.endswith('Points.mat'):
        points_path = os.path.join(naparm_path, file)
    elif file == 'ToPV_AllTrials.dat' or file == 'SpiralsAllTrials.dat':
        to_pv = os.path.join(naparm_path, file)
    elif file == 'ToSLM_AllTrials.dat' or file == 'SLMAllTrials.dat':
        to_slm = os.path.join(naparm_path, file)
        
        
       
tiff_list = []
for file in os.listdir(os.path.join(naparm_path, 'PhaseMasks')):
    if file.endswith('.tif') or file.endswith('.tiff'):
        tiff_list.append(os.path.join(naparm_path, 'PhaseMasks', file))
        

mask_list = [tifffile.imread(tiff) for tiff in tiff_list]
pm = ParseMarkpoints(xml_path=xml_path, gpl_path=gpl_path)      


In [5]:
s = ml.load_mat_struct(points_path)

points = s['points']

all_x_galvo = points.GroupCentroidX
all_y_galvo = points.GroupCentroidY
groupings = points.Group
num_groups = max(groupings)

galvo_x = []
galvo_y = []


for group in range(1,num_groups+1):
    
    group_idx = np.where(groupings==group)
    
    # xna dy galvo positions of all points in pixels
    x_px = all_x_galvo[group_idx]
    y_px = all_y_galvo[group_idx]
    
    # all elements should be the same (stolen from stack overflow)
    assert list(x_px).count(x_px[0]) == len(x_px)
    assert list(y_px).count(y_px[0]) == len(y_px)
    
    #into foxy ratio format
    galvo_x.append(x_px[0] / 512)
    galvo_y.append(y_px[0] / 512)

assert len(galvo_x) == len(galvo_y) == len(pm.durations) == len(pm.laser_powers)
num_groups = len(galvo_x)

In [7]:
#assumes that there is a trigger at t=0
num_stims = len(np.where(np.diff(pv_arr) > 0)[0]) + 1
n_repeats = num_stims / num_groups 
assert n_repeats.is_integer()
n_repeats = int(n_repeats)

10

In [8]:
mp_strings = []

#currently only supporting evenly spaced groups
inter_group_interval = slm_diff[0]

spiral_size = yaml_dict['spiral_size'] / (yaml_dict['FOVsize_UM_1x'] / yaml_dict['zoom'])
spiral_revolutions = pm.spiral_revolutions[0]

for group in range(num_groups):
    x = galvo_x[group]
    y = galvo_y[group]
    #duration = pm.durations[group]
    duration = 25
    #laser_power = pm.laser_powers[group]
    num_spirals = pm.repetitions[group]
    
    mp_string = pm.build_strings(X=x,Y=y,duration=duration,laser_power=laser_power, \
                                 is_spiral=True, spiral_size=spiral_size, \
                                 spiral_revolutions=spiral_revolutions, num_spirals=num_spirals)

    mp_strings.append(mp_string)
    

    
    
all_groups_string = pm.groups_strings(inter_group_interval=inter_group_interval, \
                                      group_list=mp_strings, SLM_trigger=True, n_repeats=n_repeats)

In [9]:
count = 0
splitter = all_groups_string.split(' ')
for s in splitter:
    if s == 'Uncaging':
        count += 1
count

400

In [10]:
num_groups

4

In [11]:
n_repeats

10

In [12]:
num_spirals

'10'

In [13]:
1/0

ZeroDivisionError: division by zero

In [14]:
sdk = SLMsdk()
sdk.SLM_connect()
mask_pointers = sdk.precalculate_and_load_first(mask_list, num_repeats=n_repeats)

SLM connection established
First mask loaded


In [15]:
slm_thread = Thread(target=sdk.load_precalculated_triggered, args = [mask_pointers])
slm_thread.start()

Ready to trigger


In [16]:
pi = PrairieInterface()

PV connection successful


In [17]:

pi.pl.SendScriptCommands(all_groups_string)


True

Trigger recieved, loaded mask 1
Trigger recieved, loaded mask 2
Trigger recieved, loaded mask 3
Trigger recieved, loaded mask 4
Trigger recieved, loaded mask 5
Trigger recieved, loaded mask 6
Trigger recieved, loaded mask 7
Trigger recieved, loaded mask 8
Trigger recieved, loaded mask 9
Trigger recieved, loaded mask 10
Trigger recieved, loaded mask 11
Trigger recieved, loaded mask 12
Trigger recieved, loaded mask 13
Trigger recieved, loaded mask 14
Trigger recieved, loaded mask 15
Trigger recieved, loaded mask 16
Trigger recieved, loaded mask 17
Trigger recieved, loaded mask 18
Trigger recieved, loaded mask 19
Trigger recieved, loaded mask 20
Trigger recieved, loaded mask 21
Trigger recieved, loaded mask 22
Trigger recieved, loaded mask 23
Trigger recieved, loaded mask 24
Trigger recieved, loaded mask 25
Trigger recieved, loaded mask 26
Trigger recieved, loaded mask 27
Trigger recieved, loaded mask 28
Trigger recieved, loaded mask 29
Trigger recieved, loaded mask 30
Trigger recieved, l

In [ ]:
all_groups_string

In [ ]:
from pyfiglet import Figlet
f = Figlet(font='invita')
print (f.renderText('AUTO NAPARM'))

In [ ]:
all_groups_string

In [ ]:
class auto_naparm:
    
    def __init__(self, naparm_path):
        self.naparm_path = naparm_path
        self.yaml_dict = yaml_dict
        
        

        
        
    






def get_yaml():
    with open(r'C:\Users\User\Documents\Code\blimp\blimp_settings.yaml', 'r') as stream:
                self.yaml_dict = yaml.load(stream)
            
               

In [ ]:
class P:

    def __init__(self,x):
        self.x = x

    @property
    def x(self):
        return self.__x

    @x.setter
    def x(self, x):
        if x < 0:
            self.__x = 0
        elif x > 1000:
            self.__x = 1000
        else:
            self.__x = x